# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Constants


MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)


In [ ]:
# # Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

In [15]:
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [11]:
def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-OCR',
 '/MiniMaxAI/MiniMax-M2',
 '/tencent/HunyuanWorld-Mirror',
 '/PaddlePaddle/PaddleOCR-VL',
 '/krea/krea-realtime-video',
 '/models',
 '/spaces/khang119966/DeepSeek-OCR-DEMO',
 '/spaces/enzostvs/deepsite',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/akhaliq/veo3.1-fast',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces',
 '/datasets/HuggingFaceFW/finewiki',
 '/datasets/karpathy/fineweb-edu-100b-shuffle',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/nick007x/github-code-2025',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly'

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'About', 'url': 'https://huggingface.co'},
  {'type': 'Company', 'url': 'https://github.com/huggingface'},
  {'type': 'Brand', 'url': 'https://www.huggingface.co/brand/'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co/'},
  {'type': 'Changelog',
   'url': 'https://huggingface.readthedocs.io/en/latest/#changelogs'},
  {'type': 'Inference', 'url': 'https://endpoints.huggingface.co/'},
  {'type': 'Spaces', 'url': 'https://huggingface.co/spaces/'},
  {'type': 'Datasets', 'url': 'https://huggingface.co/datasets/'},
  {'type': 'Learning', 'url': 'https://learn.huggingface.co/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'}]}

In [16]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'company page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'economic futures page',
   'url': 'https://www.anthropic.com/economic-futures'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'},
  {'type': 'products page', 'url': 'https://claude.com'},
  {'type': 'Claude documentation', 'url': 'https://docs.claude.com/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
3 days ago
•
1.02M
•
2.07k
MiniMaxAI/MiniMax-M2
Updated
1 day ago
•
28.4k
•
540
tencent/HunyuanWorld-Mirror
Updated
3 days ago
•
16.2k
•
365
PaddlePaddle/PaddleOCR-VL
Updated
4 days ago
•
18.7k
•
1.12k
meituan-longcat/LongCat-Video
Updated
about 2 hours ago
•
376
•
172
Browse 1M+ models
Spaces
Running
on
Zero
311
311
DeepSeek OCR Demo
🆘
An interactive demo for the DeepS

In [19]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'brand page', 'url': 'https://huggingface.co'}, {'type': 'about company', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github homepage', 'url': 'https://github.com/huggingface'}, {'type': 'twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\n3 days ago\n•\n1.02M\n•\n2.07k\nMiniMaxAI/MiniMax-M2\nUpdated\n1 day ago\n•\n28.4k\n•\n540\ntencent/HunyuanWorld-Mirror\nUpdated\n3 days ago\n•\n16.2k\n•\n365\nPaddlePaddle/PaddleOCR-VL\nUpdated\n4 days ago\n•\n18.7k\n•\n1.12k\nmeituan-longcat/LongCat-Video\nUpdated\nabout 2 hours ago\n•\n376\n•\n172\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n311\n311

In [24]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers/page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'status update', 'url': 'https://status.huggingface.co/'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}]}


**Hugging Face: Building a Better Future with AI**

**Mission**
We are the AI community building the future. Our mission is to empower machine learning developers and researchers to build, discover, and collaborate on ML models, datasets, and applications.

**Our Journey**
Founded in 2016 by Sean Li and Yiftach Baruch, Hugging Face has grown into a leading open-source platform for machine learning. We are committed to providing the tools and resources needed to accelerate innovation and progress in the field of artificial intelligence.

**Community-driven Approach**
We believe that AI should be collaborative and accessible to everyone. Our platform is designed to facilitate the sharing of knowledge, models, and datasets among researchers, developers, and organizations worldwide. With over 1 million available models, our community has made significant contributions to various ML applications, including natural language processing (NLP), computer vision, and robotics.

**Featured Applications and Models**
Explore a vast gallery of AI applications, featuring cutting-edge models such as DeepSeek-OCR, MiniMax-M2, and HunyuanWorld-Mirror. Our platform also hosts over 400k+ pre-trained models for popular deep learning frameworks like PyTorch and TensorFlow.

**Core Values**

*   **Collaboration**: We believe that collective effort leads to greater progress in AI research.
*   **Transparency**: Open-source software and data promote integrity and trust among the community.
*   **Innovation**: Embracing new ideas, technologies, and methodologies drives innovation in AI.

**Support for Business and Organization Needs**
We offer customizable Compute Plans (starting at $20/user/month), Enterprise Solutions, and Access to 45,000+ models from leading AI providers.

### Join our Community

Discover, contribute, collaborate with Hugging Face. Register now for free and unlock full access to [www.huggingface.co.](http://www.huggingface.co).

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'economic futures page', 'url': 'https://www.anthropic.com/economic-futures'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}]}


**Brochure: Anthropic**
=====================================

**Introduction**

Anthropic is a public benefit corporation dedicated to securing the benefits of artificial intelligence (AI) while mitigating its risks. Our mission is to build AI that serves humanity's long-term well-being.

**Our Mission and Values**

At Anthropic, we believe that designing powerful technologies requires both bold steps forward and intentional pauses to consider the effects. We focus on building tools with human benefit at their foundation, like Claude. Through our daily research, policy work, and product design, we aim to show what responsible AI development looks like in practice.

**Core Views on AI Safety**

We have three core views on AI safety:

* **Safety is a moral imperative**: We believe that the creation and deployment of advanced technologies like AI have significant implications for society.
* **AI should be designed with safety in mind from the start**: We prioritize human benefit at the foundation of our projects, ensuring that we build tools that align with our values.
* **Continuous research and testing are essential**: We invest in ongoing research and testing to improve our models and ensure they meet the highest standards for safety.

**Our Products**

We offer a range of products and tools designed to support safe AI development:

* **Claude**: Our flagship model, Claude is an agent that puts performance first. The best model in the world for agents, coding, and computer use.
* **Sonnet**: Sonnet offers more control over code generation and editing, providing a powerful toolset for developers.
* **Haiku**: Haiku provides advanced tools for managing context on our platform.
* **Opus**: Opus is designed to help you write and test reliable models in one go.

**Our Commitments**

We are committed to responsible scaling policies, transparency, security, compliance, and ongoing research to improve the AI landscape.

**Join Our Community**

Want to help us build the future of safe AI? Check out our current open roles on [link]. We're passionate about finding talented team members who share our values and mission.

---

*   Visit our website at [www.anthropic.ai](http://www.anthropic.ai) for more information.
*   Connect with us on social media to stay up-to-date on our latest developments.

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>